In [1]:
import os
import pickle
import numpy as np
from sklearn import decomposition, model_selection, neural_network

In [2]:
CAF_DOSE = 200

FEATURE_PATH = 'C:\\Users\\Philipp\\Documents\\Caffeine\\Features{dose}\\Combined'.format(dose=CAF_DOSE)

STAGES = ['AWA', 'NREM', 'REM']

In [3]:
with open(os.path.join(FEATURE_PATH, 'data_avg.pickle'), 'rb') as file:
    data = pickle.load(file)
with open(os.path.join(FEATURE_PATH, 'labels_avg.pickle'), 'rb') as file:
    y = pickle.load(file)
with open(os.path.join(FEATURE_PATH, 'groups_avg.pickle'), 'rb') as file:
    groups = pickle.load(file)

In [7]:
x = {}
features = []
for stage in STAGES:
    x[stage] = []
    for feature in data[stage].keys():
        features.append(feature)
        x[stage].append(data[stage][feature])
        
    x[stage] = np.concatenate(x[stage], axis=1)
    
for stage in STAGES:
    pca = decomposition.PCA(n_components=20)
    x[stage] = pca.fit_transform(x[stage])

In [8]:
searches = {}
for stage in STAGES:
    print(f'Training classifier on {stage}...')
    clf = neural_network.MLPClassifier(max_iter=1000)
    '''grid = {'hidden_layer_sizes': [(128,), (64,),
                                   (256, 64), (128, 64), (64, 32),
                                   (128, 64, 32), (256, 128, 64)],
            'activation': ['relu', 'tanh']}'''
    grid = {'hidden_layer_sizes': [(32,),
                                   (32, 16), (64, 32),
                                   (64, 32, 16), (32, 16, 8)],
            'activation': ['relu', 'tanh']}
    searches[stage] = model_selection.GridSearchCV(estimator=clf,
                                                   param_grid=grid,
                                                   scoring='accuracy',
                                                   cv=10,
                                                   iid=False,
                                                   n_jobs=-1)
    searches[stage].fit(x[stage], y[stage], groups=groups[stage])

Training classifier on AWA...


C:\Users\Philipp\Anaconda3\envs\mne\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training classifier on NREM...
Training classifier on REM...


In [9]:
for stage in STAGES:
    print(f'{stage:4}: {searches[stage].best_score_ * 100:.2f}%', searches[stage].best_params_)

AWA : 55.00% {'activation': 'tanh', 'hidden_layer_sizes': (64, 32)}
NREM: 61.67% {'activation': 'relu', 'hidden_layer_sizes': (32, 16)}
REM : 56.25% {'activation': 'relu', 'hidden_layer_sizes': (32, 16, 8)}
